## Testing single object mesh creation (optimal quality not a concern for this deliverable)

In [4]:
import math
import open3d as o3d
import pandas as pd
import numpy as np
import color_palettes as cp
import os

def object_to_mesh(fp):
    
    def change_background_to_black(vis):
        opt = vis.get_render_option()
        opt.background_color = np.asarray([0, 0, 0])
        return False
    
    key_to_callback = {}
    key_to_callback[ord("K")] = change_background_to_black

    # file reading
    pcd = o3d.io.read_point_cloud(fp)
    # normal estimation
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    # ball pivoting mesh
    point_cloud_array = np.asarray(pcd.points)
    min_xyz = point_cloud_array.min(axis=0)
    max_xyz = point_cloud_array.max(axis=0)
    min_x, min_y, min_z = min_xyz
    max_x, max_y, max_z = max_xyz
    try:
        max_dim = round(math.sqrt((max_x-min_x)**2+(max_y-min_y)**2+(max_z-min_z)**2))
        start_radius = round(max_dim**(1/16)/max_dim, 2)
        stop_radius = round(max_dim**(5/8))
    except:
        start_radius = 0.1
        stop_radius = 3
    radii = np.linspace(start=start_radius, stop=stop_radius, num=25)
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radii))
    
    out_fp = fp[:-4] + "_mesh.ply"
    o3d.io.write_triangle_mesh(out_fp, mesh)


In [ ]:
base_dir = 'Objects'

# List all objects (files and subdirectories) in the directory
objects = os.listdir(base_dir)

# iterating through each scene object to create a mesh
for obj in objects:
    if obj != ".ipynb_checkpoints": 
        obj_path = os.path.join(base_dir, obj)
        object_to_mesh(obj_path)

In [ ]:
vis = o3d.visualization.Visualizer()
vis.create_window()

for obj in objects:
    if obj[-9:] == "_mesh.ply":
        curr_mesh = o3d.io.read_triangle_mesh(obj)
        vis.add_geometry(curr_mesh)

# Adjust the camera view or perspective (optional)
view_control = vis.get_view_control()
view_control.set_lookat([0, 0, 0])  # Set the look-at point
view_control.set_up([0, 0, 1])     # Set the up vector

# Display the scene
vis.run()
vis.destroy_window()
